# 1-Environment's setup and cuda compilation check

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
!git clone https://github.com/reza-sedighi/DRU.git

In [ ]:
pwd

In [ ]:
cd DRU/

In [ ]:
!pip install -r requirements.txt

In [ ]:
cd ./models/ops

In [ ]:
!sh ./make.sh

In [ ]:
# unit test (should see all checking is True)
!python test.py

In [ ]:
cd /content/DRU

# 2-Downloading datasets

In [ ]:
import os
import requests
from tqdm import tqdm
import getpass
import zipfile

# Prompt for username and password
username = input("Enter your Cityscapes username: ")
password = getpass.getpass("Enter your Cityscapes password: ")

# Define login URL
login_url = 'https://www.cityscapes-dataset.com/login/'

# Define URLs in the specified order
dataset_urls = [
    ('https://www.cityscapes-dataset.com/file-handling/?packageID=31', '/content/DRU/data/leftImg8bit_trainval_foggyDBF.zip'),
    ('https://www.cityscapes-dataset.com/file-handling/?packageID=3', '/content/DRU/data/leftImg8bit_trainvaltest.zip')
]
payload = {
    'username': username,
    'password': password,
    'submit': 'Login'
}

# Create the dataset directory if it doesn't exist
os.makedirs('data', exist_ok=True)

def download_file(session, url, dest_path):
    """Download a file with a progress bar."""
    response = session.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kilobyte

    # Use tqdm to create a progress bar
    with open(dest_path, 'wb') as f, tqdm(
        desc=dest_path,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(block_size):
            f.write(data)
            bar.update(len(data))

def unzip_and_remove(zip_path, extract_to):
    """Unzip the file and remove the zip file."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(zip_path)  # Remove the zip file after extracting

# Start a session to manage cookies and authentication
with requests.Session() as session:
    # Post login data
    post = session.post(login_url, data=payload)

    # Check if login was successful by verifying if the session is authenticated
    if post.url == login_url:
        print("Login failed. Please check your username and password.")
    else:
        for url, zip_path in dataset_urls:
            # Download the dataset, unzip it, and remove the zip file
            download_file(session, url, zip_path)
            unzip_and_remove(zip_path, '/content/DRU/data/cityscapes')

        print("All datasets downloaded, unzipped, and cleaned up successfully.")


# 3-Separating foggy_cityscapes' images with beta=0.02

In [ ]:
import os
import shutil

# Cities split into train, val, and test
train_cities = [
    "aachen", "bochum", "bremen", "cologne", "darmstadt", "dusseldorf", 
    "erfurt", "hamburg", "hanover", "jena", "krefeld", "monchengladbach", 
    "strasbourg", "stuttgart", "tubingen", "ulm", "weimar", "zurich"
]

val_cities = ["frankfurt", "lindau", "munster"]

# test_cities = ["berlin", "bielefeld", "bonn", "leverkusen", "mainz", "munich"]

# Paths to the original and new directories
base_path = "/content/DRU/data/cityscapes/leftImg8bit_foggyDBF"
new_base_path = "/content/DRU/data/foggy_cityscapes/leftImg8bit_foggy"

# Function to handle the file moving process
def move_images(set_type, cities):
    original_set_path = os.path.join(base_path, set_type)
    new_set_path = os.path.join(new_base_path, set_type)
    
    # Create the new directory for train, val, test
    os.makedirs(new_set_path, exist_ok=True)
    
    # Loop through each city folder within the current set_type
    for city in cities:
        original_city_path = os.path.join(original_set_path, city)
        new_city_path = os.path.join(new_set_path, city)
        
        # Create the corresponding city directory in the new location
        os.makedirs(new_city_path, exist_ok=True)
        
        # Check if the city directory exists in the original path
        if os.path.exists(original_city_path):
            # Loop through each image in the city folder
            for image_name in os.listdir(original_city_path):
                # Move only images containing "_beta_0.02" in their name
                if "_beta_0.02" in image_name:
                    original_image_path = os.path.join(original_city_path, image_name)
                    new_image_path = os.path.join(new_city_path, image_name)
                    
                    # Move the image to the new directory
                    shutil.move(original_image_path, new_image_path)
                    print(f"Moved: {original_image_path} -> {new_image_path}")
        else:
            print(f"City folder not found: {original_city_path}")

# Process each set (train, val, test) with the correct cities
move_images("train", train_cities)
move_images("val", val_cities)
# move_images("test", test_cities)


# 4-Download coco-style annotations and put them into correct directory

In [ ]:
# please add a shortcut of weights and annotations to your google drive then try to mount your gdrive in order to download the files
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# after downloading the coco-style annotations put them into th correct directory based on the following organization, note that data_root is /content/DRU/data


# [data_root]
# └─ cityscapes
# 	└─ annotations
# 		└─ cityscapes_train_cocostyle.json
# 		└─ cityscapes_train_caronly_cocostyle.json
# 		└─ cityscapes_val_cocostyle.json
# 		└─ cityscapes_val_caronly_cocostyle.json
# 	└─ leftImg8bit
# 		└─ train
# 		└─ val
# └─ foggy_cityscapes
# 	└─ annotations
# 		└─ foggy_cityscapes_train_cocostyle.json
# 		└─ foggy_cityscapes_val_cocostyle.json
# 	└─ leftImg8bit_foggy
# 		└─ train
# 		└─ val
# └─ sim10k
# 	└─ annotations
# 		└─ sim10k_train_cocostyle.json
# 	└─ JPEGImages
# └─ bdd10k
# 	└─ annotations
# 		└─ bdd100k_daytime_train_cocostyle.json
# 		└─ bdd100k_daytime_val_cocostyle.json
# 	└─ images

# 5-Downloading trained weights

In [ ]:
# please download the trained weights according to the readme.md file and then put them in /content/DRU/outputs/def-detr-base/city2foggy/source_only & teaching_standard & teaching_mask1 respectively

All experiments are conducted with batch size 8 (for source_only: 8 labeled samples; for teaching_standard or teaching_mask: 8 unlabeled samples), on an NVIDIA Quadro RTX 8000 GPU (48GB).

city2foggy: Cityscapes → FoggyCityscapes(level 0.02)


In [ ]:
cd /content/DRU/weights

# 6-Training and Evaluation

## 6.1 Training

In [ ]:
# First, run source_only to pretrain the Source-only model. Then, run teaching_standard to train the conventional Mean-Teacher framework OR teaching_mask to train the proposed DRU.

# For example in city2foggy benchmark, first edit the files in configs/def-detr-base/city2foggy/ to specify your own DATA_ROOT and OUTPUT_DIR, then run:

# sh configs/def-detr-base/city2foggy/source_only.sh
# sh configs/def-detr-base/city2foggy/teaching_standard.sh
# sh configs/def-detr-base/city2foggy/teaching_mask.sh

## 6.2 Evaluation


In [ ]:
# To evaluate the trained model and get the predicted results, run:

# sh configs/def-detr-base/city2foggy/evaluation_source_only.sh
# sh configs/def-detr-base/city2foggy/evaluation_teaching_standard.sh
# sh configs/def-detr-base/city2foggy/evaluation_teaching_mask.sh